In [ ]:
import numpy as np
import pandas as pd
import statistics as st

import matplotlib
import matplotlib.pyplot as plt
import statistics as st

matplotlib inline

from astropy.io import fits
from astropy import units as u
from astropy import constants as const
from astropy.visualization import quantity_support
quantity_support()
from astropy.nddata import StdDevUncertainty

from specutils import Spectrum1D

######################################### creating files below ####################################################

c_u = (const.c.value *1e6) *(u.micron/u.s)   # units = [um/s]
CO_file = '/Users/temprecover/Dropbox/Michael_Sanchez_project/COCO.xlsx' # Contains CO RoVib wavelengths

COlines_table = pd.read_excel(CO_file, sheet_name="COlineranges")     # excel sheet in which info is contained

########################################## opening files below ####################################################

# .fits files defined as ffile[0] & ffile[1] for data and stacked line profiles, respectively.

# x = fromfile(fname, dtype=dat)
ffile = [
        ['/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/RULup_M_APR07.fits',     #     [ 0 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/AATau_M.fits',           #     [ 1 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/CVCha_M.fits',           #     [ 2 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/DFTAU_M.fits',           #     [ 3 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/DoAr44_M_SEP07.fits',    #     [ 4 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/DRTau_M_DEC08.fits',     #     [ 5 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/EXLup14_M.fits',         #     [ 6 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/HD135344B_M_APR07.fits', #     [ 7 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/HD142527_M_5AUG08.fits', #     [ 8 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/HD144432S_M.fits',       #     [ 9 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/TWHya_M.fits',           #    [ 10 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/RYLup_M_APR07.fits',     #    [ 11 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/HD36112_ISHELL.fits',    #    [ 12 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/TWCha_M_JAN09.fits',     #    [ 13 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/HD139614_Carmona.fits'], #    [ 14 ]

        ['/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/RULup_COv1-0stacked.fits',     #     [ 0 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/AATau_COv1-0stacked.fits',     #     [ 1 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/CVCha_COv1-0stacked.fits',     #     [ 2 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/DFTAU_COv1-0stacked.fits',     #     [ 3 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/DoAr44_COv1-0stacked.fits',    #     [ 4 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/DRTau_COv1-0stacked.fits',     #     [ 5 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/EXLup14_COv1-0stacked.fits',   #     [ 6 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/HD135344B_COv1-0stacked.fits', #     [ 7 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/HD142527_COv1-0stacked.fits',  #     [ 8 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/HD144432S_COv1-0stacked.fits', #     [ 9 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/TWHya_COv1-0stacked.fits',     #    [ 10 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/RYLup_COv1-0stacked.fits',     #    [ 11 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/HD36112_COv1-0stacked.fits',   #    [ 12 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/TWCha_COv1-0stacked.fits',     #    [ 13 ]
         '/Users/temprecover/Dropbox/Michael_Sanchez_project/DATA/stacked_lines/HD139614_COv1-0stacked.fits']  #    [ 14 ]
        ]

#defining column headers to create Pandas dataframe object
flhead = ['RULup_M_APR07', 'AATau_M','CVCha_M','DFTAU_M','DoAr44_M_SEP07','DRTau_M_DEC08','EXLup14_M','HD135344B_M_APR07',
          'HD142527_M_5AUG08','HD144432S_M','TWHya_M','RYLup_M_APR07','HD36112_ISHELL','TWCha_M_JAN09','HD139614_Carmona']
errhead = ['RULup_M_APR07_err','AATau_M_err','CVCha_M_err','DFTAU_M_err','DoAr44_M_SEP07_err','DRTau_M_DEC08_err',
           'EXLup14_M_err','HD135344B_M_APR07_err','HD142527_M_5AUG08_err','HD144432S_M_err','TWHya_M_err',
           'RYLup_M_APR07_err','HD36112_ISHELL_err','TWCha_M_JAN09_err','HD139614_Carmona_err']
heads=flhead+errhead
################################ Isolating columns from excel spreadsheet: COlines_table ################################
COl = COlines_table['line']
COw = COlines_table['wl']
COv = COlines_table['vib']
COb = COlines_table['branch']
coo = np.where( (COv=='v1-0') & (COb=='P') )
P   = coo[0]

# Isolating (v = 1->0) and (P-lines) from COlines_table. Below are the indexes
#P=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,
#   23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43]

fluxdf = pd.DataFrame(columns=heads, index=COl[P])  # Dataframe object to be filled with line fluxes & their error

for y in range(len(heads)):
    fluxdf[heads[y]] = np.repeat(0., len(P))         # Populating each column with 0's to fill in later.

print(fluxdf[heads[:5]][:10])


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NEW CELL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NEW CELL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NEW CELL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NEW CELL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NEW CELL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NEW CELL ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

win  = 5.2e-4*u.um  # range within continuum values will be measured/fitted
cont = 1.95e-3      # distance from central line at which 'win' will begin

# To save plots to exported pdfs (March 19 was the last time I saved plots as of this parenthetical note, 03/24/2020)
#pdf_disks = PdfPages('2020-03-19 Plines_vs_Fluxes.pdf')
#pdf_spectra = PdfPages('2020-03-19 All Disk Spectra.pdf')

for f in range(len(ffile[0])): #'|||||||||||||||||||||| THUS, THE FOR LOOP BEGINS ||||||||||||||||||||'

    specdata = fits.open(ffile[0][f])[1].data[0]   # opens .fits files
    fits.open(ffile[0][f]).close()                 # Following online astropy documentation, this opens then closes .fits file hdu dat
    wv  = specdata[0]*1e-3                         # This and the following two lines are accessing the wavelength, flux density, and uncertainty arrays from the .fits file, respectively.
    fln =(specdata[1])
    err = StdDevUncertainty(specdata[2], unit = 'Jy')
    m_spec = Spectrum1D(flux=fln*u.Jy, spectral_axis=wv*u.micron, uncertainty=err)  # Spectrum1D() comes from the specutils package and I use it to hold each spectrum.

    # Later in the code, I further split this master Spectrum1D (m_spec) into subspectral objects that identify individual lines in the spectrum (sub_spec objects)

    # Creating empty arrays to later fill in to then add to the all-zero pandas datafram object: fluxdf.
    flux_col = []   # Calculated line fluxes go here
    flerr_col = []  # Calculated line flux errors go here
    ind_used = []   # I am storing the indexes (P-lines) used from each line for reference.

    # I implement an offset to be applied to almost all the disks in our sample.
    # This offset is in microns and is a measure of how far off the central peak of the line is from the expected value in CO_linestable.
    offset = 0.0

    if f == 0:    # RULup
        offset = 4e-5
    elif f== 1:   # AATau
        offset = 1e-4
    elif f== 3:   # DFTau
        offset = 2e-4
    elif f== 4:   # DoAr
        offset = 1.5e-4
    elif f== 5:   # DRTau
        offset = 1.8e-4
    elif f== 7:   # HD-44B
        offset = 1.2e-4
    elif f== 8:   # HD-2527
        offset = 1e-4
    elif f== 9:   # HD-32S
        offset = 5e-5
    elif f== 10:  # TWHya
        offset = 4.5e-5
    elif f== 11:  # RYLup
        offset = 6e-5
    elif f== 12:  # HD-112 ISHELL
        offset = -1.6e-4
    elif f== 13:  # TWCha
        offset = 1e-4
    elif f== 14:  # HD-9614
        offset = 4e-5

    # Below, I am running the same procedure as above to the stacked line profiles.
    stackdata = fits.open(ffile[1][f])[1].data[0]
    fits.open(ffile[1][f]).close()
    vel = stackdata[0] # km/s
    flx = stackdata[1]
    errr = stackdata[2]
    yfitt = stackdata[3]
    '---------------------------------- PDF EXPORT/SAVE ----------------------------------------------'
    #pdf_lines = PdfPages('P_lines-'+flhead[f]+'.pdf')

    '====================================== LINE FLUX CALCULATION LOOP BEGIN ======================================'
    '====================================== LINE FLUX CALCULATION LOOP BEGIN ======================================'
    '====================================== LINE FLUX CALCULATION LOOP BEGIN ======================================'
####
    for spectum_name in offset_dict.key():
    ####
        'OFFSET -------|-------'
        CO_w = COw[i] + offset_dict[spectum_name]

        ' GLOBAL CONTINUUM BOUNDARY ASSIGNMENTS  --------[-]--^--[-]--'

        ileft = CO_w - cont
        left = ileft - win.value
        iright = CO_w + cont
        right = iright + win.value

        ' SPECTRA BOUNDARIES  [--^-^-^^--^-^-^^--^-] '

        if left < min(m_spec.spectral_axis.value):
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif right > max(m_spec.spectral_axis.value):
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==14 and i>=15: # CARMONA SPECTRA CLIPPING STOP AT P15
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue

        ' GLOBAL LINE BOUNDARY ASSIGNMENTS  ------------[-^-]----'
        ' CONTINUUM/LINE BOUNDARY SHIFTS  ----[-]-_-[-^]-[-]------'

        # This little bit of adjustment is a relic of previous attempts at defining the continuum and line boundaries.
        # It took less time to determine those values this way with what I had already done instead of re-finding the correct boundary value for each line.
        k = (cont/win.value)     # k = 3.75 -n
        l_line = CO_w - (k-1)*win.value  # k = 2.75 for original line boundary value, k-1
        r_line = CO_w + (k-1)*win.value  # (k-1)*win.value= cont - win.value

        # Below are the adjusted line & continuum boundary conditions for each line in every disk.
        if f==0:   #RULup

            if i == (2)+1: # 'P2'
                l_line = CO_w - 1.75*win.value
            elif i == (3)+1:
                r_line = CO_w + cont - 1.5*win.value
            elif i == (5)+1: # P5
                l_line = CO_w - 1.75*win.value
                #ileft...
                left = ileft - win.value
            elif i == (7)+1: # P7
                r_line = CO_w + 2.55*win.value
                iright = CO_w + cont + 0.5*win.value
                right = iright + win.value
            elif i == (9)+1: # P9
                r_line = CO_w + (k-2)*win.value
            elif i == (13)+1: # P13
                iright = CO_w + 2*cont
                right = iright + win.value
            elif i == (14)+1: # P14
                iright = CO_w + 0.95*cont
                right = iright + win.value
            elif i == (15)+1: # P15
                iright = CO_w + cont + win.value
                right = iright + win.value
            elif i == (21)+1: # P21
                l_line = CO_w - 2.25*win.value
            elif i == (23)+1:
                ileft = CO_w - 2*cont
                left = ileft - win.value

        elif f==1:   #AATau

            if i == (3)+1: # Previously - COl[i] == 'P3'
                r_line = CO_w + 2.25*win.value
            elif i == (7)+1: # P7
                r_line = CO_w + 2.1*win.value
                iright = CO_w + cont + 0.5*win.value
                right = iright + win.value
            elif i == (15)+1: # P15
                iright = CO_w + cont + win.value
                right = iright + win.value
            elif i == (17)+1: # 17
                ileft = CO_w - 1.3*cont
                left = ileft - win.value

        elif f==2:   #CVCha
            r_line = CO_w + (k-1.7)*win.value
            l_line = CO_w - (k-1.7)*win.value

            if i == (31)+1: #P31
                ileft = CO_w - 2*cont
                left = ileft - win.value
                iright = CO_w + 2*cont
                right = iright + win.value

        elif f==3:   #DFTau
            l_line = CO_w - (k-2)*win.value
            r_line = CO_w + (k-2)*win.value

            if i == (2)+1: #P2
                l_line = CO_w - (k-2.4)*win.value
                ileft = CO_w - 1.2*cont
                left = ileft - win.value
                iright = CO_w + 0.9*cont
                right = iright + win.value
            elif i == (4)+1: #P4
                l_line = CO_w - (k-2.2)*win.value
            elif i == (5)+1: #P5
                #l_line = CO_w - 1*win.value
                ileft = CO_w - 2*cont
                left = ileft - win.value
                iright = CO_w + 1.6*cont
                right = iright + win.value


        elif f==4:   #DoAr44
            l_line = CO_w - (k-1.9)*win.value
            r_line = CO_w + (k-1.9)*win.value

            if i == (2)+1: # P2
                l_line = CO_w - (k-2)*win.value
            elif i == (3)+1:  # P3
                l_line = CO_w - (k-2.4)*win.value
                r_line = CO_w + (k-2.4)*win.value
            elif i == (5)+1: # P5
                r_line = CO_w + (k-2.2)*win.value
            elif i == (11)+1:
                r_line = CO_w + (k-2.5)*win.value

        elif f==5:   #DRTau
            l_line = CO_w - (k-2.4)*win.value
            r_line = CO_w + (k-2.4)*win.value

            if i == (2)+1: # P2
                r_line = CO_w + (k-3)*win.value
                l_line = CO_w - (k-2.7)*win.value
            elif i == (3)+1: # P3
                l_line = CO_w - (k-2.7)*win.value
            elif i == (8)+1:
                l_line = CO_w - (k-3)*win.value
            elif i == (9)+1: # P9
                l_line = CO_w - (k-3.1)*win.value
            elif i == (26)+1: # P26
                l_line = CO_w - (k-2.6)*win.value
            elif i == (29)+1: # P29
                r_line = CO_w + (k-2.8)*win.value
                l_line = CO_w - (k-2.8)*win.value
            elif i == (32)+1: # P32
                l_line = CO_w - (k-2.6)*win.value

        elif f==6:   #EXLup
            l_line = CO_w - (k-1.5)*win.value
            r_line = CO_w + (k-1.5)*win.value
            #lin = cont - 1.5*win.value
            if i == (4)+1: # P2
                r_line = CO_w + (k-2.5)*win.value
            elif i == (21)+1: # P21
                r_line = CO_w + (k-2.4)*win.value
            elif i == (22)+1: # P22
                r_line = CO_w + (k-2.3)*win.value


        elif f==7:   #HD135344B - previously 2.4*win
            l_line = CO_w - (k-3.2)*win.value
            r_line = CO_w + (k-3.2)*win.value

            if i == (2)+1: # P2
                r_line = CO_w + (k-3.4)*win.value
            elif i == (3)+1: # P3
                r_line = CO_w + (k-3.4)*win.value
            elif i == (23)+1: # P23
                l_line = CO_w - (k-3.4)*win.value
            elif i == (27)+1: # P27
                iright = CO_w + 0.5*cont
                right = iright + win.value
            elif i == (31)+1: # P31
                ileft = CO_w - 0.3*cont
                left = ileft - win.value
                iright = CO_w + 0.3*cont
                right = iright + win.value
            elif i == (32)+1: # P32
                ileft = CO_w - 0.5*cont
                left = ileft - win.value
                iright = CO_w + 0.5*cont
                right = iright + win.value

        elif f==8:   #HD142527 - 2.6*win
            l_line = CO_w - (k-3)*win.value
            r_line = CO_w + (k-3)*win.value

        elif f==9:   #HD144432S - 2*win
            l_line = CO_w - (k-2.5)*win.value
            r_line = CO_w + (k-2.5)*win.value

            if i == (5)+1: # 'P5'
                l_line = CO_w - (k-2.8)*win.value
            elif i == (6)+1: # P6
                l_line = CO_w - (k-3.1)*win.value
            elif i == (7)+1: # P7
                l_line = CO_w - (k-3.1)*win.value
                iright = CO_w + 1.5*cont
                right = iright + win.value
            elif i == (8)+1: # P8
                l_line = CO_w - (k-3.1)*win.value
            elif i == (9)+1: # P9
                l_line = CO_w - (k-2.9)*win.value
            elif i == (10)+1: # P10
                l_line = CO_w - (k-2.9)*win.value


        elif f==10:   #TWHya - 2.5
            l_line = CO_w - (k-3.5)*win.value
            r_line = CO_w + (k-3.5)*win.value

            ileft = CO_w - .5*cont
            left = ileft - win.value
            iright = CO_w + .5*cont
            right = iright + win.value

            if i == (1)+1:
                l_line = CO_w - (k-3.65)*win.value

        elif f==11:   #RYLup 2.5
            l_line = CO_w - (k-3)*win.value
            r_line = CO_w + (k-3)*win.value

            if i ==(14)+1: # P14
                ileft = CO_w - 1.3*cont
                left = ileft - win.value
            elif i == (7)+1: # P7
                l_line = CO_w - (k-3.2)*win.value
            elif i == (8)+1: # P8
                l_line = CO_w - (k-3.1)*win.value
                r_line = CO_w + (k-3.2)*win.value
            elif i == (10)+1: # P10
                r_line = CO_w + (k-3.2)*win.value
            elif i == (11)+1: # P11
                r_line = CO_w + (k-3.4)*win.value
            elif i == (13)+1: # P13
                r_line = CO_w + (k-3.3)*win.value
            elif i == (14)+1: # P14
                r_line = CO_w + (k-3.2)*win.value
            elif i == (17)+1: # P17
                r_line = CO_w + (k-3.3)*win.value
            elif i == (20)+1: # P20
                r_line = CO_w + (k-3.4)*win.value
            elif i == (26)+1: # P26
                r_line = CO_w + (k-3.2)*win.value
            elif i == (27)+1: # P27
                iright = CO_w + 2*cont
                right = iright + win.value

        elif f==12:   #ISHELL - 2*win
            l_line = CO_w - (k-3.2)*win.value
            r_line = CO_w + (k-3.2)*win.value

            if i == (1)+1: # P1
                r_line = CO_w +(k-3.3)*win.value
            elif i == (2)+1: # P2
                r_line = CO_w +(k-3.4)*win.value
            elif i == (3)+1: # P3
                r_line = CO_w +(k-3.3)*win.value
            elif i == (24)+1: # P24
                r_line = CO_w + (k-3.3)*win.value
            elif i == (31)+1: # P31
                ileft = CO_w - 1.2*cont
                left = ileft - win.value
                iright = CO_w + .5*cont
                right = iright + win.value
            elif i == (34)+1: # P34
                iright = CO_w + .3*cont
                right = iright + .7*win.value
            elif i == (35)+1: # P35
                iright = CO_w + 1.2*cont
                right = iright + win.value
            elif i == (39)+1: # P39
                iright = CO_w + .3*cont
                right = iright + win.value

        elif f==13:   #TWCha
            l_line = CO_w - (k-2)*win.value
            r_line = CO_w + (k-2)*win.value

            if i == (10)+1: # P10
                l_line = CO_w - (3)*win.value
            elif i == (11)+1: # P11
                l_line = CO_w - (k-2.5)*win.value
            elif i == (21)+1: # P21
                r_line = CO_w + (k-3.3)*win.value
            elif i == (31)+1: # P31
                ileft = CO_w - 2*cont
                left = ileft - win.value
                iright = CO_w + 2*cont
                right = iright + win.value

        elif f==14:   #Carmona - 2*win
            l_line = CO_w - (k-3.35)*win.value
            r_line = CO_w + (k-3.35)*win.value

            if i == (13)+1: # P13 - reducing cont. bounds -> eliminates > 5 Jy data.
                iright = CO_w + .2*cont
                right = iright + .5*win.value
            #elif i == (11)+1:
            #    ileft = CO_w - .75*cont
            #    left = CO_w - 1.5*win.value
            #    iright = CO_w + .75*cont
            #    right = CO_w + 1.5*win.value


        ' LINE OMISSIONS  -----^-----[    ]---^--'
#
        if f==0 and i==2: ########## RULup - P1
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==5 and i==23: ########## DRTau - P22
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==5 and i==24: ########## DRTau - P23
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==5 and i==25: ########## DRTau - P24
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==6 and i==3: ########## EXLup - P2
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==6 and i==4: ########## EXLup - P3
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==11 and i==28: ########## RYLup - P27
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==12 and i==33:  ########## ISHELL - P32
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==12 and i==37:  ########## ISHELL - P36
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==13 and i==18:  ########## TWCha - P17
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==14 and i==9: ########## Carmona - P8
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==14 and i==17: ########## Carmona P16*
            print('P16 isolated')
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif f==14 and i==26: ########## Carmona P25 (???? CARMONA HAS NO 25..? maybe ISHELL?)
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue

        '############################################### SUB_SPEC OBJECT #################################################'
        p_ind = np.where(np.logical_and(m_spec.spectral_axis.value>=left, m_spec.spectral_axis.value<=right))
        sub_spec = Spectrum1D(flux=m_spec.flux[p_ind], spectral_axis=m_spec.spectral_axis[p_ind],
                              uncertainty=m_spec.uncertainty[p_ind])
        sub_spec_err = specdata[2][p_ind] # Ideally 1% of flux density

        '############################################ 0 & infs -> NaNs ############################################'
        if sub_spec.flux.value.any() < 0.0: #? what is this? <0.0*u.Jy: ## Aimed toward ISHELL data: large negative values
            sub_spec.flux[(sub_spec.flux<0.*u.Jy)] = np.nan
            #print('values less than 0 Jy Detected',sub_spec.flux[(sub_spec.flux<0.*u.Jy)])

        elif np.any(abs(np.isinf(sub_spec.flux))) == True:
            sub_spec.flux[np.where(np.isinf(sub_spec.flux))] = np.nan
            #print('Infinities detected',sub_spec.flux[np.isinf(sub_spec.flux)])

        '################################## left line < LINE BOOL < right line #####################################'
        line_bool = (sub_spec.spectral_axis > l_line*u.micron) & (sub_spec.spectral_axis < r_line*u.micron)

        '############################################# Isolating NaNs ##################################################'
        ynan_lb = sub_spec.flux[np.where(np.isnan(sub_spec.flux[line_bool]))]
        xnan_lb = sub_spec.spectral_axis[np.where(np.isnan(sub_spec.flux[line_bool]))]
        fl_nans = sub_spec.flux[np.where(np.isnan(sub_spec.flux))]
        wv_nans = sub_spec.spectral_axis[np.where(np.isnan(sub_spec.flux))]
        fl_num = len(sub_spec.flux[line_bool])-len(ynan_lb)
        '''                               FILTERING BAD LINES W/ 1:3 NAN RATIO                                   '''
        if fl_num == 0.: # Is it more effecient/direct to say this -> if len(sub_spec.flux)==0.:?
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue
        elif len(ynan_lb)/len(sub_spec.flux[line_bool]) >= 3.3/10:
            flux_col.append(0.0)
            flerr_col.append(0.0)
            continue

        '########################################## INTERPOLATING DATA ###############################################'
        tf_no = np.isnan(sub_spec.flux)
        minterp = np.interp(wv_nans, sub_spec.spectral_axis[~tf_no], sub_spec.flux[~tf_no]) # Interpolating all sub_spec
        sub_spec.flux[np.where(np.isnan(sub_spec.flux))] = minterp # There is an Error here for [14:15]

        '###################### INTERPOLATING UNCERTAINTY VALUES WITH REPEATED MEDIAN VALUES #############################'
        if np.any(np.isinf(sub_spec_err)) == True:
            sub_spec_err[np.where(np.isinf(sub_spec_err))] = np.median(sub_spec_err[~np.isnan(sub_spec_err)])
            #print('infinities in uncertainty')
        if np.any(np.isnan(sub_spec_err)) == True:
            sub_spec_err[np.where(np.isnan(sub_spec_err))] = np.median(sub_spec_err[~np.isnan(sub_spec_err)])
            #print('nans in uncertainty')

        '########################################## cont_bool isolates indexes of continuum outside of peak: #########'
        ######################################### left < cont_nool < ileft &&& iright > cont_nool > right #############
        cont_bool = (sub_spec.spectral_axis < ileft*u.micron) | (sub_spec.spectral_axis > iright*u.micron)
        uncert_cb = sub_spec_err[cont_bool]


        '#################################### Polynomial Continuum Fitting #############################'
        p_fit = np.polyfit(sub_spec.spectral_axis.value[cont_bool],sub_spec.flux.value[cont_bool],
                           deg=1, w=(uncert_cb)**(-2), cov='unscaled')# unscaled means our uncertainties are reliable.
        b_par = p_fit[0][0]
        a_par = p_fit[0][1]
        a_err = np.sqrt(np.diag(p_fit[1])[1]) # stdev a
        b_err = np.sqrt(np.diag(p_fit[1])[0]) # stdev b
        y_continuum_fitted = a_par + ( b_par*sub_spec.spectral_axis.value ) # y=a+bx Continuum Fit
        ###### IMPLEMENT CONTINUUM SLOPE IF-STATEMENT TO FLAG POTENTIALLY BAD LINES AND
        ###### REDUCE RESTRICTIONS IN ISOLATION CONDITIONS ABOVE WITH CONTINUUM SLOPE.

        '############################################# Line Flux calculation #############################################'
        dx = c_u/(sub_spec.spectral_axis) # Frequency Conversion
        a_dx = abs(np.diff(dx))
        avg = st.mean(a_dx.value)
        avg_dx = np.repeat(avg, len(sub_spec.spectral_axis[line_bool]))*u.Hz  # (px size in freq. about the same overall)
        flux = (sub_spec.flux - y_continuum_fitted*u.Jy)
        '                                           Final Line Flux calculation '
        net_flux = np.sum(flux[line_bool] * avg_dx)
        net_flux_erg=net_flux.to(u.erg * u.cm**-2 * u.s**-1)

        '############################################## ERROR ANALYSIS ###################################################'
        ############################################## ERROR ANALYSIS ###################################################
        conterr = np.sqrt((a_err**2) + (sub_spec.spectral_axis.value*(b_err))**2) # Jansky
        F_i =  sub_spec_err + conterr   # Also Janksy
        Flerr_ = np.sqrt(np.sum((avg_dx*F_i[line_bool])**2))
        Flerr = (Flerr_*u.Jy).to(u.erg*u.cm**-2 * u.s**-1) # Chng of units: Jy*Hz -> erg/(s*cm2)

        '++++++++++++++++++++++++++++++++++++++++++ Saving Flux+Error Values ++++++++++++++++++++++++++++++++++++++++++'
        if (net_flux_erg.value) < 0.0:
            print('negative flux value detected')
            flux_col.append(0.0)
        else:
            flux_col.append(net_flux_erg.value)

        flerr_col.append(Flerr.value)
        ind_used.append(i-2) #Corresponds to correct indexes for fluxdf.index
    ####
        '################################################# PLOTTING LINES #################################################'
        ################################################## PLOTTING LINES #################################################

        if i>=0: # LOOP = i singling. I used this to plot specific lines as opposed to all of them in one disk.
            #print(i,COl[i],'Err:Flux ratio_target(~.01) =',Flerr/net_flux_erg)
    ########
            FIG = plt.figure(figsize = (14,7))
            plt.grid(True, which='major')
            plt.minorticks_on()
            plt.grid(True, which='minor',linestyle='--')
            plt.xlim(left-.001,right+.001)
            #plt.ylim(min(sub_spec.flux[line_bool])-.1*u.Jy,max(sub_spec.flux[line_bool])+.01*u.Jy)
            plt.plot(sub_spec.spectral_axis, sub_spec.flux, color = 'black', linestyle='-')
            plt.plot( ((vel/(const.c.value*1e-3))+1)*COw[i], yfitt,
                     color = 'red', linestyle='--') #### STACKED LINE PROFILe
            plt.errorbar(sub_spec.spectral_axis, sub_spec.flux, yerr=specdata[2][p_ind]*u.Jy,
                         fmt=' ', errorevery=10, ms=10)
            plt.plot(sub_spec.spectral_axis,(a_par + ( b_par*sub_spec.spectral_axis.value )), color='orange')
            plt.plot(wv_nans,fl_nans, color='green', linestyle=':')
            plt.title(COl[i])
            plt.fill_between(sub_spec.spectral_axis[line_bool], sub_spec.flux[line_bool],
                             y_continuum_fitted[line_bool], color='gray') ######## Hopefully a viable fill color #########
            for c in COw[P]:
                plt.axvline(x=c, color='blue', linestyle='--')
            plt.axvline(x=CO_w, color='green', linestyle=':')
            plt.axvline(x=left, color='red', linestyle=':')
            plt.axvline(x=right, color='red', linestyle=':')
            plt.axvline(x=ileft, color='red', linestyle=':')
            plt.axvline(x=iright, color='red', linestyle=':')
            plt.axvline(x=l_line, color='red', linestyle='-.')
            plt.axvline(x=r_line, color='red', linestyle='-.')
        #pdf_lines.savefig(FIG)
    ################################################### END LINE LOOP ################################################
    ################################################### END LINE LOOP ################################################
    ################################################### END LINE LOOP ################################################
    '--------------------------Closing PdfPages instance-----------------------'
    #pdf_lines.close()
    '########################## APPENDING VALUES TO APPROPRIATE DATAFRAME COLUMNS ######################################'
    fluxdf[flhead[f]] = flux_col
    fluxdf[errhead[f]] = flerr_col

    '################################################# PLOTTING LINES #################################################'
    ############################################## P vs Flux plot ####################################################

    # P vs FLUX Plots
    pf_fig = plt.figure(figsize=(21,7))
    plt.grid(True)
    plt.title(flhead[f])
    plt.plot(fluxdf[flhead[f]].index, np.log10(flux_col),'.')
    yerr = (0.434 * (fluxdf[errhead[f]]/flux_col))
    plt.errorbar(fluxdf[flhead[f]].index, np.log10(flux_col), yerr=yerr,fmt=' ', ms=10)
    plt.xlabel('P-Line')
    plt.ylabel('ln(Line Flux)')

    #pdf_disks.savefig(pf_fig)

    ######################################### FULL SPECTRA (with nans) ############################################

    sp_fig = plt.figure(figsize=(50,10))
    plt.title(flhead[f])
    for j in P:
        plt.axvline(x=COw[j], color='green', linestyle='--', label=COl[j])
        #plt.text(y,s,COl[j])
    plt.xlim(min(m_spec.spectral_axis)-(0.01*u.um),max(m_spec.spectral_axis)+(0.01*u.um))
    #plt.ylim(min(m_spec.flux[np.where(~np.isnan(m_spec.flux))]),2)
    plt.plot(wv, fln) # Also want to plot interpolated values over data separately in one plot.

    #pdf_spectra.savefig(sp_fig)

    '########################################## END ALL DISK LOOP ################################################'
    '########################################## END ALL DISK LOOP ################################################'
    '########################################## END ALL DISK LOOP ################################################'
    print('End')
################# THE MAIN EXPORT: FLUXDF DATAFRAME WITH LINE FLUXES + ERRORS

#pdf_disks.close()
#pdf_spectra.close()
' FIN '